Import thư viện

In [122]:
import torch
import torch.nn as nn
from torchvision import datasets
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets.utils import download_url
import os
import tarfile
import matplotlib.pyplot as plt
import numpy as np


Tải dữ liệu

In [123]:
#dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
##with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    #tar.extractall(path='./data_encoder')

train_ds=datasets.CIFAR10(root='../Deep-Learning-Models/data/cifar10',train= True, transform=transforms.ToTensor())
test_ds=datasets.CIFAR10(root='../Deep-Learning-Models/data/cifar10',train= False, transform=transforms.ToTensor())

batch_size=128
train_loader=DataLoader(train_ds,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_ds,batch_size=batch_size,shuffle=False)

Sử dụng GPU

In [124]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [125]:
def ConvLayer(inp, out, ks=3, s=1, p=1):
    return nn.Conv2d(inp,out,kernel_size=ks,stride=s,padding=p)
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.neural_net = nn.Sequential (
            ConvLayer(3, 32), nn.ReLU(),
            ConvLayer(32, 64), nn.ReLU(),
            nn.MaxPool2d(2, 2), # Output: (batch_size, 64, x/2, x/2)
            ConvLayer(64, 128), nn.ReLU(),
            ConvLayer(128, 256), nn.ReLU(),
            nn.MaxPool2d(2, 2), # Output: (batch_size, 256, x/4, x/4)
            ConvLayer(256, 512), nn.ReLU(),
            ConvLayer(512, 1024), nn.ReLU(),
            nn.MaxPool2d(2, 2), # Output: (batch_size, 1024, x/8, x/28
            ConvLayer(1024, 2048), nn.ReLU(),
            nn.MaxPool2d(2, 2), # Output: (batch_size, 2048, x/16, x/16)
            nn.Flatten(), # Output: (batch_size, 2048 * 2 * 2) được huấn luyện trên cifar10
            nn.Linear(2048 * 2 * 2, 1024), nn.ReLU(),
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, 10),
    )
    def forward(self, x):
        return self.neural_net(x)

In [126]:
model = CNN().to(device)

Hàm đánh giá

In [127]:
def accurate_score(out, labels):
    _, preds = torch.max(out, dim=1)
    correct_preds = torch.sum(preds == labels).item()
    total_preds = len(preds)
    accuracy = torch.tensor(correct_preds / total_preds)
    return accuracy

Training model

In [128]:
num_epochs = 10
learning_rate = 0.003
loss_func= nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_testing_accuracies = []
    for img, labels in train_loader:
        img =img.to(device)
        labels = labels.to(device)
        pred = model(img)
        loss = loss_func(pred, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    for test_img, test_labels in test_loader:
        test_img = test_img.to(device)
        test_labels = test_labels.to(device)
        test_pred = model(test_img)
        metrics = accurate_score(test_pred, test_labels)
        total_testing_accuracies.append(metrics)
    print("Epoch Number: ", epoch, "Testing Accuracy Score: ", sum(total_testing_accuracies)/len(total_testing_accuracies))

Epoch Number:  0 Testing Accuracy Score:  tensor(0.3243)
Epoch Number:  1 Testing Accuracy Score:  tensor(0.4276)
Epoch Number:  2 Testing Accuracy Score:  tensor(0.5186)
Epoch Number:  3 Testing Accuracy Score:  tensor(0.5757)
Epoch Number:  4 Testing Accuracy Score:  tensor(0.5955)
Epoch Number:  5 Testing Accuracy Score:  tensor(0.6358)
Epoch Number:  6 Testing Accuracy Score:  tensor(0.6492)
Epoch Number:  7 Testing Accuracy Score:  tensor(0.6624)
Epoch Number:  8 Testing Accuracy Score:  tensor(0.6706)
Epoch Number:  9 Testing Accuracy Score:  tensor(0.6687)


Lưu mô hình

In [129]:
torch.save(model.state_dict(), 'CNN.pth')

In [150]:
# tải mô hình
state_dict = torch.load("./CNN.pth")
print(type(state_dict))
for key, value in state_dict.items():
    print(f"{key}:")
    print(value)
    print("-" * 50)

C:\Users\admin\AppData\Local\Temp\ipykernel_20100\3702514272.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("./CNN.pth")


<class 'collections.OrderedDict'>
neural_net.0.weight:
tensor([[[[-0.0510,  0.0303,  0.2066],
          [-0.1941,  0.0828,  0.2001],
          [-0.1677, -0.0909,  0.0883]],

         [[ 0.1502, -0.2233, -0.0328],
          [-0.2915,  0.0369,  0.0882],
          [-0.2771,  0.1571,  0.3094]],

         [[ 0.0543, -0.1954, -0.0872],
          [-0.0141, -0.1608,  0.1092],
          [-0.1782,  0.0097,  0.2493]]],


        [[[-0.0424, -0.0841, -0.0374],
          [-0.1157,  0.1524, -0.0968],
          [ 0.0848,  0.0805,  0.1587]],

         [[-0.2086, -0.0454, -0.0523],
          [-0.2487,  0.1497, -0.0265],
          [-0.1443,  0.0227, -0.0719]],

         [[ 0.1427,  0.1421,  0.2069],
          [ 0.1671,  0.0791,  0.0519],
          [ 0.1942, -0.0144, -0.0949]]],


        [[[ 0.1072, -0.0574, -0.1866],
          [-0.1815,  0.0755,  0.0209],
          [ 0.1705, -0.0775, -0.1918]],

         [[ 0.0326,  0.1007,  0.0172],
          [-0.1728, -0.1168, -0.1045],
          [ 0.0660, -0.1741, -

Test encoder

In [149]:
class EncoderCNN(nn.Module):
    def __init__(self):
        super(EncoderCNN, self).__init__()
        self.CNN = CNN()
        
        state_dict = torch.load("./CNN.pth", map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
        self.CNN.load_state_dict(state_dict)

        modules = list(self.CNN.neural_net.children())[:-6]
        print(len(modules))

        self.CNN = nn.Sequential(*modules)

    def forward(self, images):
        print("Input images shape:", images.shape)  # In kích thước đầu vào
        for idx, layer in enumerate(self.CNN):
            images = layer(images)  # Áp dụng từng lớp lên đầu vào
            print(f"Layer {idx}: {layer} -> Output shape: {images.shape}")  # In kích thước đầu ra của từng lớp
        return images
    
# Test lớp EncoderCNN
encoder = EncoderCNN().to(device)

# Ví dụ với đầu vào ảnh giả lập
images = torch.randn(1, 3, 224, 224).to(device)
print("Partial values (5x5):", images[:, :, :5, :5])  # Chỉ in một phần tensor

features = encoder(images)
print("Extracted Features Shape:", features.shape)
print("Extracted Features Tensor:")
print(features)


C:\Users\admin\AppData\Local\Temp\ipykernel_20100\1809718159.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("./CNN.pth", map_location=torch.devi

18
Partial values (5x5): tensor([[[[-0.4196, -1.7226,  0.2972, -0.4058,  0.7685],
          [ 0.9914, -0.8200,  0.4358, -0.4330,  0.8913],
          [-0.3248, -2.0333, -0.6716,  0.3098,  0.7064],
          [ 0.0289, -1.3407,  0.4622,  1.3046,  0.8801],
          [-0.6841,  1.4287,  0.5942,  0.4188, -1.8508]],

         [[-1.5648, -0.7283, -0.2702,  0.7977, -0.5072],
          [ 0.9987, -0.3806,  1.4947,  0.6487, -0.3929],
          [-1.3738, -2.7185,  0.2250, -0.5122,  0.4356],
          [ 0.0269,  1.9484,  0.0151,  0.1573, -1.1159],
          [-0.2287, -1.4467,  0.3909,  0.2613,  0.0075]],

         [[ 0.3854,  0.3109,  0.0361, -1.1079, -1.4616],
          [ 0.4000,  1.1072, -0.0687, -0.0647,  2.9482],
          [-1.3308,  0.5920, -0.3383,  1.3653,  0.1145],
          [-0.5146, -0.5531,  0.3356, -0.2534, -0.2240],
          [-0.3412, -0.3388, -0.4695,  0.5649, -0.0447]]]], device='cuda:0')
Input images shape: torch.Size([1, 3, 224, 224])
Layer 0: Conv2d(3, 32, kernel_size=(3, 3), stri